# Imports & Prepare Data

In [1]:
from models.ml_sequential import prepare_data, run_all
import tensorflow as tf
# Disable GPU training, comment out to enable
tf.config.set_visible_devices([], 'GPU')
# Imports used across models
from keras.layers import Input, Dense, TimeDistributed, Dropout
from keras.models import Sequential
from keras.regularizers import l2

features = ['female', 'age', 'height', 'mass', 'ta_set', 'rh_set']
features_scaler, output_scaler, X_padded, y_padded, max_len = prepare_data(features)

# Define the input/output shape
input_shape = (None, X_padded.shape[-1])
output_shape = y_padded.shape[-1]

# Train the model with early stopping
from keras.callbacks import EarlyStopping

/Users/s5068337/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Max sequence length: 600


# LSTM

In [2]:
from keras.layers import LSTM

# MODEL NAME
model_name = 'ml_lstm'

# Model architecture
model = Sequential()
model.add(Input(shape=input_shape))
model.add(LSTM(16, return_sequences=True, dropout=0.2, recurrent_dropout=0.2, kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01)))
model.add(TimeDistributed(Dense(8, activation='relu', kernel_regularizer=l2(0.01))))
model.add(Dropout(0.2))
model.add(TimeDistributed(Dense(output_shape)))

# Compile the model
model.compile(loss='mse', optimizer='adam')

# Train the model
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
model.fit(X_padded, y_padded, validation_split=0.2, epochs=100, batch_size=32, callbacks=[early_stopping])

# Save the model
model.save('model_weights/{}.keras'.format(model_name))

# Run simulations using trained model
run_all(model, model_name, features, features_scaler, output_scaler, max_len)

Epoch 1/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 22s 2s/step - loss: 0.7826 - val_loss: 0.8579
Epoch 2/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 138ms/step - loss: 0.7339 - val_loss: 0.7763
Epoch 3/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 170ms/step - loss: 0.6884 - val_loss: 0.7039
Epoch 4/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 155ms/step - loss: 0.6602 - val_loss: 0.6467
Epoch 5/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 155ms/step - loss: 0.6007 - val_loss: 0.5990
Epoch 6/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 143ms/step - loss: 0.5893 - val_loss: 0.5539
Epoch 7/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 146ms/step - loss: 0.5302 - val_loss: 0.5129
Epoch 8/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 147ms/step - loss: 0.5015 - val_loss: 0.4762
Epoch 9/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 140ms/step - loss: 0.4721 - val_loss: 0.4449
Epoch 10/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 162ms/step - loss: 0.4631 - val_loss: 0.4181
Epoch 11/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 143ms/step - loss: 0.4429 - val_loss: 0.3950
Epoch 12/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 162ms/step - loss: 0.416

# GRU

In [4]:
from tensorflow.keras.layers import GRU

# MODEL NAME
model_name = 'ml_gru'

# Model architecture
model = Sequential()
model.add(Input(shape=input_shape))
model.add(GRU(16, return_sequences=True))
model.add(TimeDistributed(Dense(8, activation='relu', kernel_regularizer=l2(0.01))))
model.add(Dropout(0.2))
model.add(TimeDistributed(Dense(output_shape)))

# Compile the model
model.compile(loss='mse', optimizer='adam')

# Train the model
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
model.fit(X_padded, y_padded, validation_split=0.2, epochs=100, batch_size=32, callbacks=[early_stopping])

# Save the model
model.save('model_weights/{}.keras'.format(model_name))

# Run simulations using trained model
run_all(model, model_name, features, features_scaler, output_scaler, max_len)

Epoch 1/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 23s 2s/step - loss: 0.6427 - val_loss: 0.5698
Epoch 2/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 168ms/step - loss: 0.6176 - val_loss: 0.5441
Epoch 3/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 145ms/step - loss: 0.5973 - val_loss: 0.5194
Epoch 4/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 148ms/step - loss: 0.5514 - val_loss: 0.4953
Epoch 5/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 146ms/step - loss: 0.5327 - val_loss: 0.4718
Epoch 6/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 149ms/step - loss: 0.5059 - val_loss: 0.4493
Epoch 7/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 148ms/step - loss: 0.4718 - val_loss: 0.4283
Epoch 8/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 146ms/step - loss: 0.4532 - val_loss: 0.4087
Epoch 9/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 150ms/step - loss: 0.4258 - val_loss: 0.3900
Epoch 10/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 146ms/step - loss: 0.4066 - val_loss: 0.3718
Epoch 11/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 153ms/step - loss: 0.3824 - val_loss: 0.3544
Epoch 12/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 158ms/step - loss: 0.355

# TCN

In [5]:
from tensorflow.keras.layers import Conv1D

# MODEL NAME
model_name = 'ml_tcn'

model = Sequential()
model.add(Input(shape=input_shape))
model.add(Conv1D(16, kernel_size=3, activation='relu', padding='causal', kernel_regularizer=l2(0.01)))
model.add(TimeDistributed(Dense(8, activation='relu', kernel_regularizer=l2(0.01))))
model.add(Dropout(0.2))
model.add(TimeDistributed(Dense(output_shape)))

model.compile(loss='mse', optimizer='adam')

early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
model.fit(X_padded, y_padded, validation_split=0.2, epochs=100, batch_size=32, callbacks=[early_stopping])

model.save('model_weights/{}.keras'.format(model_name))

# Run simulations using trained model
run_all(model, model_name, features, features_scaler, output_scaler, max_len)

Epoch 1/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 22s 2s/step - loss: 0.8685 - val_loss: 0.7256
Epoch 2/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step - loss: 0.7624 - val_loss: 0.6452
Epoch 3/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step - loss: 0.6980 - val_loss: 0.5776
Epoch 4/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step - loss: 0.6099 - val_loss: 0.5250
Epoch 5/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step - loss: 0.5476 - val_loss: 0.4865
Epoch 6/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step - loss: 0.4985 - val_loss: 0.4537
Epoch 7/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step - loss: 0.4675 - val_loss: 0.4245
Epoch 8/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step - loss: 0.4354 - val_loss: 0.3993
Epoch 9/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step - loss: 0.4068 - val_loss: 0.3768
Epoch 10/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step - loss: 0.3895 - val_loss: 0.3561
Epoch 11/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step - loss: 0.3724 - val_loss: 0.3369
Epoch 12/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step - loss: 0.3564 - val_los

# Seq2Seq

In [2]:
from tensorflow.keras.layers import LSTM, Attention, Concatenate
from tensorflow.keras.models import Model

# MODEL NAME
model_name = 'ml_seq2seq'

encoder_inputs = Input(shape=input_shape)
encoder_lstm = LSTM(16, return_sequences=True, return_state=True, dropout=0.2, recurrent_dropout=0.2, kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01))
encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)

decoder_inputs = Input(shape=input_shape)
decoder_lstm = LSTM(16, return_sequences=True, return_state=True, dropout=0.2, recurrent_dropout=0.2, kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01))
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=[state_h, state_c])

attention = Attention()([decoder_outputs, encoder_outputs])
decoder_concat = Concatenate()([decoder_outputs, attention])
decoder_dense = TimeDistributed(Dense(output_shape))
decoder_outputs = decoder_dense(decoder_concat)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='adam', loss='mse')

early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
model.fit([X_padded, X_padded], y_padded, validation_split=0.2, epochs=100, batch_size=32, callbacks=[early_stopping])

model.save('model_weights/{}.keras'.format(model_name))

# Run simulations using trained model
run_all(model, model_name, features, features_scaler, output_scaler, max_len, is_transformer=True)

Epoch 1/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 11s 1s/step - loss: 1.0749 - val_loss: 0.8671
Epoch 2/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 263ms/step - loss: 0.9825 - val_loss: 0.7839
Epoch 3/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 249ms/step - loss: 0.8935 - val_loss: 0.7124
Epoch 4/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 249ms/step - loss: 0.7871 - val_loss: 0.6514
Epoch 5/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 267ms/step - loss: 0.6984 - val_loss: 0.5998
Epoch 6/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 238ms/step - loss: 0.6609 - val_loss: 0.5575
Epoch 7/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 277ms/step - loss: 0.6086 - val_loss: 0.5249
Epoch 8/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 239ms/step - loss: 0.5649 - val_loss: 0.5019
Epoch 9/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 250ms/step - loss: 0.5363 - val_loss: 0.4870
Epoch 10/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 257ms/step - loss: 0.5199 - val_loss: 0.4781
Epoch 11/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 254ms/step - loss: 0.4849 - val_loss: 0.4722
Epoch 12/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 248ms/step - loss: 0.477